<a href="https://colab.research.google.com/github/a9e68ce5/Machine-Learning/blob/main/SVM%20by%20handcraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.optim import SGD
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

seed = 3047
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

In [ ]:
!gdown 1o0m3jyfmetUOJ146TqHuEGUWwQyC7JXV
!gdown 1B5OC3R0yM8F7yjoYOKu3t08QZalcr7DC
!gdown 1THvOuf_EOn6c_6TLy0Bqs23BP2NraBR2

Downloading...
From: https://drive.google.com/uc?id=1o0m3jyfmetUOJ146TqHuEGUWwQyC7JXV
To: /content/train.csv
100% 6.54M/6.54M [00:00<00:00, 45.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1B5OC3R0yM8F7yjoYOKu3t08QZalcr7DC
To: /content/val.csv
100% 665k/665k [00:00<00:00, 117MB/s]
Downloading...
From: https://drive.google.com/uc?id=1THvOuf_EOn6c_6TLy0Bqs23BP2NraBR2
To: /content/X_test
100% 3.57M/3.57M [00:00<00:00, 278MB/s]


In [ ]:
features=["age", 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss']

In [ ]:
class SVM(nn.Module):
  def __init__(self):
    # TODO design your model
    super(SVM, self).__init__()
    self.w = nn.Parameter(torch.randn((1, 1024)).to(torch.float32))#outi∼N(0,1)
    self.f = nn.Sequential(
                  nn.Linear(107, 128),
                  nn.Dropout(0.5),
                  nn.LeakyReLU(negative_slope=0.05),
                  nn.Linear(128,256),
                  nn.LeakyReLU(negative_slope=0.05),
                  nn.Linear(256,512),
                  nn.LeakyReLU(negative_slope=0.05),
                  nn.Linear(512,1024),
                )
  def transform(self, x):
    x = self.f(x)
    return x
  def kernel(self, x):
    pass
  def forward(self, x):
    f = torch.matmul(self.transform(x), self.w.T)

    return f

In [ ]:
class HingeLoss(nn.Module):
  def __init__(self, C):
    super(HingeLoss, self).__init__()
    self.C = C
  def forward(self, y, f):
    #y=y.detach().cpu().numpy()
    #f=f.detach().cpu().numpy()
    loss=0.0
    one = torch.ones(len(y)).cuda()
    two = (2 * one).cuda()
    l = torch.sub(one , torch.mul(y , f)).cuda()
    lt = torch.div(torch.add(torch.sign(l),one),two).cuda()
    loss = torch.sum(torch.mul(l,lt)).cuda() # define Hinge loss
    return self.C * loss


In [ ]:
class TrainDataset(Dataset):
  def __init__(self, split, mu=None, std=None):
    X = pd.read_csv(f"{split}.csv")

    Y = X['y'].values.reshape(-1) * 2 - 1
    X = X.drop(labels=['y'],axis=1)#106
    X_nodummy = self.normalize(X, mu, std)
    X_dummy = X.drop(features,axis=1)
    f = [X_nodummy,X_dummy]
    X = pd.concat(f,axis=1)
    X = np.concatenate((X, np.ones((X.shape[0], 1))), 1)
    self.Y = torch.from_numpy(Y).to(torch.float32)
    self.X = torch.from_numpy(X).to(torch.float32)

  def normalize(self, X, mu=None, std=None):
    mu_x = X[features].mean()
    std_x = X[features].std()
    X = (X[features]-mu_x)/std_x

    return X

  def __len__(self):
    return self.X.size(0)

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]

class TestDataset(Dataset):
  def __init__(self, mu=None, std=None):
    X = pd.read_csv("X_test")
    X_nodummy = self.normalize(X, mu, std)
    X_dummy = X.drop(features,axis=1)
    f = [X_nodummy,X_dummy]
    X = pd.concat(f,axis=1)
    X = np.concatenate((X, np.ones((X.shape[0], 1))), 1)
    self.X = torch.from_numpy(X).to(torch.float32)

  def normalize(self, X, mu_x=None, std_x=None):
    mu_x = X[features].mean()
    std_x = X[features].std()
    X = (X[features]-mu_x)/std_x

    return X

  def __len__(self):
    return self.X.size(0)

  def __getitem__(self, idx):
    return self.X[idx]

In [ ]:
def train(train_data, val_data, model, optim, C, device='cuda:0'):
    epoch = 100
    objective = HingeLoss(C)
    steps = 0
    best = 0

    for e in range(epoch):
      for tr in train_data:
        steps += 1
        x_train, y_train = tr
        x_train, y_train = x_train.to(device), y_train.to(device)
        pred = model(x_train).squeeze(1)
        loss = objective(pred, y_train) + 1 / 2 * torch.sum(model.w[:-1] ** 2)

        optim.zero_grad()
        loss.backward()
        optim.step()


        if steps % 100 == 0:
          model.eval()
          with torch.no_grad():
            acc = []
            for val in val_data:
              x_val, y_val = val
              x_val , y_val = x_val.to(device), y_val.to(device)
              pred = model(x_val).squeeze(1)
              pred = (pred > 0) * 2 - 1

              result = (y_val == pred)
              acc += [(float(result.sum()) / result.size(0))]
            acc = sum(acc) / len(acc)
            print(f'Steps {steps}| Train Loss = {loss.item()}| Val acc = {acc}')
            if acc > best:
              torch.save(model.state_dict(), 'best.ckpt')
              print('model saved to best.ckpt' )
              best = acc
          model.train()
    return model

In [ ]:
lr = 0.0001 #原本得太大要改小一點
batch = 32
C = 1
device = 'cuda:0'

In [ ]:
trainset = TrainDataset('train')
devset = TrainDataset('val')
testset = TestDataset()

train_dataloader = DataLoader(trainset, batch, True, drop_last=False)
val_dataloader = DataLoader(devset, 1, False)
test_dataloader = DataLoader(testset, 1, False)

model = SVM().to(device)
optim = SGD(model.parameters(), lr)
model = train(train_dataloader, val_dataloader, model, optim, C, device)

Steps 100| Train Loss = 8.749897956848145| Val acc = 0.7696666666666667
model saved to best.ckpt
Steps 200| Train Loss = 10.213069915771484| Val acc = 0.8033333333333333
model saved to best.ckpt
Steps 300| Train Loss = 12.723794937133789| Val acc = 0.8156666666666667
model saved to best.ckpt
Steps 400| Train Loss = 9.709233283996582| Val acc = 0.8116666666666666
Steps 500| Train Loss = 15.825675010681152| Val acc = 0.775
Steps 600| Train Loss = 9.970705032348633| Val acc = 0.834
model saved to best.ckpt
Steps 700| Train Loss = 10.852099418640137| Val acc = 0.826
Steps 800| Train Loss = 28.935596466064453| Val acc = 0.814
Steps 900| Train Loss = 13.653066635131836| Val acc = 0.7903333333333333
Steps 1000| Train Loss = 14.782269477844238| Val acc = 0.7996666666666666
Steps 1100| Train Loss = 16.88784408569336| Val acc = 0.8373333333333334
model saved to best.ckpt
Steps 1200| Train Loss = 14.495923042297363| Val acc = 0.8363333333333334
Steps 1300| Train Loss = 12.889062881469727| Val acc

In [ ]:
best_model = model
best_model.load_state_dict(torch.load('best.ckpt'))
best_model = best_model.eval()

y_test = []
for x in test_dataloader:
  x = x.to(device)
  y = best_model(x)
  y_test.append(((y > 0) * 1).item())



In [ ]:
import csv
with open('predict.csv', 'w', newline='') as csvf:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvf)
    writer.writerow(['id','label'])
    for i in range(len(y_test)):
      writer.writerow( [i + 1, int(y_test[i])] )